In [2]:
pip install langchain_community

In [3]:
pip install pypdf

In [26]:
#Loading the PDF file
from langchain_community.document_loaders import PyPDFLoader
pdf = PyPDFLoader("/content/No context behind paper.pdf")
print(pdf)

In [1]:
pip install langchain-google-genai

In [5]:
#generating the embeddings for the PDF file
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key,
                                               model="models/embedding-001")

In [6]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38

In [7]:
#storing the embeddings in Chromadb
from langchain_community.vectorstores import Chroma

In [8]:
db_connection=Chroma(persist_directory="./chroma_db_", embedding_function= embedding_model)

In [9]:
retriver = db_connection.as_retriever(search_kwargs={"k":5})
print(type(retriver))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [10]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [11]:
#creating chat template
chat_template = ChatPromptTemplate.from_messages([
    #System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot.
    You take the content and question from user. Your answer should be based on the specific content."""),
    #Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context:
    {context}

    Question:
    {question}
    Answer:""")
])

In [12]:
#Initializing chat model
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(google_api_key,
                                    model="gemini-1.5-pro-latest")

In [13]:
#output parsing
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [14]:
#RAG chain creation
def format_docs(docs):
  return "\n\n".join(doc.pagecontent for doc in docs)

rag_chain = (
    {"context":retriver | format_docs, "question":RunnablePassthrough()}
    |chat_template
    |chat_model
    |output_parser
)

In [27]:
#user question
response = rag_chain.invoke("can you tell me about local attention")
response

'## Local Attention Explained\n\nUnfortunately, the provided context is empty, making it impossible to give a specific explanation of local attention within a particular framework. However, I can still offer a general overview of the concept:\n\n**Local Attention** is a mechanism in attention-based models (like transformers) that focuses on a limited window of neighboring tokens instead of attending to the entire input sequence. This approach offers several advantages:\n\n* **Efficiency:** Processing only a subset of tokens significantly reduces computational cost and memory requirements compared to global attention. This makes local attention suitable for longer sequences or resource-constrained environments.\n* **Speed:**  By limiting the attention scope, calculations happen faster, leading to quicker model training and inference.\n* **Inductive Bias:** Local attention inherently encodes an inductive bias towards capturing local dependencies and relationships between words, which can

In [18]:
pip install ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00


In [20]:
from IPython.display import Markdown as md
md(response)

## Local Attention Explained

While the provided context doesn't explicitly define "local attention," I can still explain the concept based on my general knowledge.

**Local Attention** is a mechanism within the broader concept of attention mechanisms used in various sequence-based models, particularly in natural language processing (NLP). Unlike "global attention," where all elements in a sequence interact with each other, local attention focuses on a restricted window of neighboring elements for each element in the sequence.

**Here's how it works:**

1. **Limited Scope:** For each element in the sequence, a local context window is defined. This window includes the element itself and a fixed number of its neighbors on either side. 
2. **Attention Calculation:** The attention mechanism operates only within this local window, calculating attention scores between the element and its neighbors. These scores represent the importance of each neighbor for understanding the current element.
3. **Weighted Representation:** Using the attention scores, a weighted representation of the local context is created. This representation captures the relevant information from the neighboring elements.

**Benefits of Local Attention:**

* **Reduced computational cost:** Compared to global attention, local attention requires less computation as it only considers a small window of elements. This makes it more efficient for longer sequences.
* **Capturing local dependencies:** Local attention excels at capturing dependencies between nearby elements in a sequence, which is crucial for tasks like understanding the structure of sentences or relationships between words.
* **Parallel processing:** Local attention allows for parallel processing of different parts of the sequence, leading to faster computation.

**Examples of Local Attention:**

* **Transformer architectures:** Local attention is often used in Transformer models, where it can be implemented as a restricted self-attention mechanism.
* **Convolutional Neural Networks (CNNs):** CNNs inherently employ a form of local attention by using convolutional filters to extract features from local regions of the input data. 

**In conclusion, local attention provides an efficient way to capture local dependencies within sequences, making it a valuable tool in various NLP tasks and sequence-based models.** 


In [25]:
response = rag_chain.invoke("what is the attention mechanism?")
md(response)

## Understanding the Attention Mechanism

The provided context seems to be missing information about the attention mechanism. However, I can still explain what the attention mechanism is and how it works in general.

**What is the Attention Mechanism?**

The attention mechanism is a crucial component in various sequence-based tasks, particularly in natural language processing (NLP) and computer vision. Inspired by human attention, it allows models to focus on specific parts of the input sequence when processing information, enhancing their understanding and performance.

**How does it work?**

1. **Input Encoding:** The input sequence, such as a sentence or image, is first encoded into a representation that captures its features.
2. **Query, Key, and Value:** For each element in the input sequence, three vectors are generated:
    * **Query:** Represents what the model is looking for.
    * **Key:** Represents the information each element holds.
    * **Value:** Represents the actual information to be extracted from the element.
3. **Attention Scores:** The query is compared with each key to calculate attention scores, indicating the relevance of each element to the current focus.
4. **Softmax and Weighted Average:** The attention scores are normalized using a softmax function, and then used to create a weighted average of the value vectors. This weighted average represents the context-aware representation of the input sequence.
5. **Output Generation:** The context-aware representation is used to generate the output, such as a translated sentence or image caption.

**Benefits of Attention Mechanism:**

* **Improved Accuracy:** By focusing on relevant parts of the input, models can make more accurate predictions.
* **Interpretability:** Attention weights provide insights into which parts of the input the model considers important for a specific task. 
* **Handling Long Sequences:** Attention helps models handle long sequences effectively by selectively focusing on relevant information.

**Types of Attention Mechanisms:**

* **Self-Attention:** Relates different positions within the same sequence to compute a representation of the sequence.
* **Soft Attention:** Allows the model to attend to multiple parts of the input with varying degrees of importance.
* **Hard Attention:** The model focuses on only one specific part of the input at a time.

**Applications of Attention Mechanism:**

* **Neural Machine Translation:** Improves translation quality by focusing on relevant parts of the source sentence.
* **Text Summarization:** Identifies important sentences in a document to generate concise summaries.
* **Image Captioning:** Helps models focus on relevant regions in an image to generate accurate captions.
* **Speech Recognition:** Improves accuracy by focusing on relevant parts of the audio signal.

**Although the provided context is incomplete, I hope this explanation gives you a good understanding of the attention mechanism and its significance in various applications.** 
